# GIFT Spectral Gap v3: Unbiased Test

**Key improvements over v2:**
1. **Neutral initialization**: lambda starts at 1.0, NOT at 14/H*
2. **Track minimum**: Record the lowest eigenvalue found
3. **Multiple runs**: Average over random seeds

**The test**: Does lambda naturally converge to 14/H* without being told the target?

In [ ]:
!pip install -q torch numpy scipy matplotlib tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
class CurvedLaplacian:
    def __init__(self, metric_fn):
        self.metric_fn = metric_fn
        
    def __call__(self, f_net, x):
        x = x.requires_grad_(True)
        batch_size, dim = x.shape
        
        g_diag = self.metric_fn(x)
        g_inv = 1.0 / (g_diag + 1e-10)
        sqrt_det_g = torch.sqrt(torch.prod(g_diag, dim=1, keepdim=True))
        
        f = f_net(x).squeeze(-1)
        grad_f = torch.autograd.grad(f.sum(), x, create_graph=True)[0]
        
        laplacian = torch.zeros(batch_size, device=x.device)
        for i in range(dim):
            coeff = sqrt_det_g.squeeze() * g_inv[:, i]
            flux = coeff * grad_f[:, i]
            div_flux = torch.autograd.grad(flux.sum(), x, create_graph=True)[0][:, i]
            laplacian = laplacian + div_flux / (sqrt_det_g.squeeze() + 1e-10)
        
        return laplacian

In [ ]:
class G2TCSMetric:
    def __init__(self, b2, b3, vol_normalized=True):
        self.b2, self.b3 = b2, b3
        self.H_star = b2 + b3 + 1
        self.dim = 7
        self.T = np.sqrt(self.H_star)
        self.T0 = self.T / 3
        
        if vol_normalized:
            self._compute_vol_normalization()
        else:
            self.vol_factor = 1.0
    
    def _compute_vol_normalization(self, n_samples=100000):
        x = torch.rand(n_samples, 7, device=device)
        x[:, 0] *= self.T
        t = x[:, 0]
        h_val = torch.cosh((t - self.T/2) / self.T0)
        sqrt_g = h_val**6
        vol_raw = torch.mean(sqrt_g).item() * self.T
        self.vol_factor = vol_raw**(-1/7)
        
    def h(self, t):
        return torch.cosh((t - self.T/2) / self.T0)
    
    def metric(self, x):
        t = x[:, 0]
        h_val = self.h(t)
        g = torch.ones(x.shape[0], 7, device=x.device)
        g[:, 1:] = (h_val**2).unsqueeze(1).expand(-1, 6)
        return g * self.vol_factor**2
    
    def sqrt_det_g(self, x):
        t = x[:, 0]
        return self.h(t)**6 * self.vol_factor**7
    
    def sample(self, n_points):
        x = torch.rand(n_points, 7, device=device)
        x[:, 0] *= self.T
        return x
    
    def lambda_1_predicted(self):
        return 14.0 / self.H_star

In [ ]:
class EigenfunctionNet(nn.Module):
    def __init__(self, input_dim=7, hidden_dim=256, n_layers=8, fourier_features=64):
        super().__init__()
        self.register_buffer('B', torch.randn(input_dim, fourier_features) * 2.0)
        encoded_dim = input_dim + 2 * fourier_features
        
        self.input_layer = nn.Linear(encoded_dim, hidden_dim)
        self.res_blocks = nn.ModuleList([
            nn.Sequential(nn.Linear(hidden_dim, hidden_dim), nn.GELU(),
                         nn.Linear(hidden_dim, hidden_dim))
            for _ in range(n_layers)
        ])
        self.output_layer = nn.Linear(hidden_dim, 1)
        self.log_lambda = nn.Parameter(torch.tensor(0.0))  # lambda starts at 1.0
        
    def fourier_encode(self, x):
        proj = x @ self.B
        return torch.cat([x, torch.sin(proj), torch.cos(proj)], dim=-1)
    
    def forward(self, x):
        h = F.gelu(self.input_layer(self.fourier_encode(x)))
        for block in self.res_blocks:
            h = h + 0.1 * block(h)
        return self.output_layer(h)
    
    def get_lambda(self):
        return torch.exp(self.log_lambda).item()

In [ ]:
def train_unbiased(metric_obj, n_epochs=5000, n_points=10000, lr=5e-4, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    
    net = EigenfunctionNet().to(device)
    laplacian = CurvedLaplacian(metric_obj.metric)
    
    optimizer = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=500)
    
    history = {'lambda': [], 'loss_pde': []}
    best_lambda = float('inf')
    best_epoch = 0
    
    pbar = tqdm(range(n_epochs), desc="Training")
    for epoch in pbar:
        x = metric_obj.sample(n_points)
        sqrt_g = metric_obj.sqrt_det_g(x)
        
        f = net(x).squeeze(-1)
        lam = torch.exp(net.log_lambda)
        lap_f = laplacian(net, x)
        
        loss_pde = torch.mean((lap_f + lam * f)**2 * sqrt_g) / torch.mean(sqrt_g)
        f_mean = torch.sum(f * sqrt_g) / torch.sum(sqrt_g)
        f_norm = torch.sum(f**2 * sqrt_g) / torch.sum(sqrt_g)
        loss_orth = f_mean**2
        loss_norm = (f_norm - 1.0)**2
        
        loss = loss_pde + 100*loss_orth + 10*loss_norm
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        current_lambda = net.get_lambda()
        history['lambda'].append(current_lambda)
        history['loss_pde'].append(loss_pde.item())
        
        if epoch > 200 and loss_norm.item() < 0.1:
            if current_lambda < best_lambda:
                best_lambda = current_lambda
                best_epoch = epoch
        
        if epoch % 500 == 0:
            pbar.set_postfix({'lam': f"{current_lambda:.4f}", 
                            'min': f"{best_lambda:.4f}" if best_lambda < float('inf') else "..."})
    
    return {
        'lambda_final': history['lambda'][-1],
        'lambda_min': best_lambda if best_lambda < float('inf') else history['lambda'][-1],
        'best_epoch': best_epoch,
        'history': history
    }

## Main Experiment

In [ ]:
print("="*70)
print("UNBIASED TEST: lambda starts at 1.0, NOT at 14/H*")
print("="*70)

manifolds = [
    {"name": "K7 (GIFT)", "b2": 21, "b3": 77},
    {"name": "Joyce J1", "b2": 12, "b3": 43},
    {"name": "Joyce J4", "b2": 0, "b3": 103},
    {"name": "Kovalev TCS", "b2": 0, "b3": 71},
]

results = []

for m in manifolds:
    print(f"\n{'-'*50}")
    print(f"{m['name']}: H* = {m['b2'] + m['b3'] + 1}")
    print(f"GIFT prediction: 14/H* = {14/(m['b2']+m['b3']+1):.6f}")
    print(f"Initial lambda = 1.0 (NEUTRAL)")
    
    metric = G2TCSMetric(m['b2'], m['b3'])
    
    run_results = []
    for seed in [42, 123, 456]:
        print(f"\nRun seed {seed}:")
        r = train_unbiased(metric, n_epochs=4000, n_points=10000, seed=seed)
        run_results.append(r)
        print(f"  Final: {r['lambda_final']:.6f}, Min: {r['lambda_min']:.6f}")
    
    results.append({
        'name': m['name'],
        'H_star': m['b2'] + m['b3'] + 1,
        'lambda_pred': 14 / (m['b2'] + m['b3'] + 1),
        'lambda_final': np.mean([r['lambda_final'] for r in run_results]),
        'lambda_min': np.mean([r['lambda_min'] for r in run_results]),
        'runs': run_results
    })

In [ ]:
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

print(f"\n{'Manifold':<15} {'H*':>5} {'14/H*':>10} {'lam_min':>10} {'lam_final':>10}")
print("-"*55)

for r in results:
    print(f"{r['name']:<15} {r['H_star']:>5} {r['lambda_pred']:>10.6f} "
          f"{r['lambda_min']:>10.6f} {r['lambda_final']:>10.6f}")

print("\nINVARIANT: lambda_min * H* = ?")
products = [r['lambda_min'] * r['H_star'] for r in results]
for r, p in zip(results, products):
    print(f"  {r['name']}: {p:.4f}")
print(f"\nMean: {np.mean(products):.4f} (target: 14)")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, r in zip(axes.flat, results):
    history = r['runs'][0]['history']
    ax.plot(history['lambda'], 'b-', alpha=0.7, label='lambda(t)')
    ax.axhline(y=r['lambda_pred'], color='r', linestyle='--', label=f'14/H*={r["lambda_pred"]:.4f}')
    ax.axhline(y=r['lambda_min'], color='g', linestyle=':', label=f'min={r["lambda_min"]:.4f}')
    ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.3)
    ax.set_title(f"{r['name']} (H*={r['H_star']})")
    ax.set_xlabel('Epoch')
    ax.set_ylabel('lambda')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('unbiased_convergence.png', dpi=150)
plt.show()